In [ ]:
from Bio import SeqIO, Entrez
import os
from urllib.error import HTTPError
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import permutations, product
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import tqdm
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_val_score

from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
from sklearn.metrics import accuracy_score, auc, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc, roc_curve, roc_auc_score

from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier

from sklearn.model_selection import GridSearchCV

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

Entrez.tool = "Zoonosis predictor"

Entrez.email = input("Enter an email address to use NCBI e-utils: ")

# in order to import model utils and validation utils
if (os.path.abspath('').split('/')[-1] == 'project'):
    %cd utils
elif (os.path.abspath('').split('/')[-1] == 'train_and_vis'):
    %cd ../utils

import query_utils
import model_utils
import validation_utils

if (os.path.abspath('').split('/')[-1] == 'utils'):
    %cd ..

In [ ]:
def resetkmerdict(permset)->OrderedDict:
        kmerdict = OrderedDict()
        for i in permset:
            kmerdict[i]=0
        return kmerdict

def assign_kmers_to_dict(st, permset, kmer):
    kmerdict=resetkmerdict(permset)
    # st = row[2] # tune for which column the sequence is in
    for j in range(len(st)-kmer+1):
        if not st[j:j+kmer] in permset: continue
        kmerdict[st[j:j+kmer]]+=1
    return kmerdict

def getTrainParams(mergedDf, kmer):

    s = product('acgt',repeat = kmer)
    permset = set(["".join(x) for x in list(s)])
    # print(permset)

    l = []
    
    for row in tqdm.tqdm(mergedDf.itertuples()):
        # print(row)
        l.append(assign_kmers_to_dict(row[1], permset, kmer))

    finalkmerdict=pd.DataFrame(l)
    # print(finalkmerdict)

    # print("finished")
    # mergedDf.fillna(0, inplace=True)

    X = finalkmerdict
    X = X.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)
    Y = mergedDf['isZoonotic']

    

    # print(X)
    # print(Y)
    return (X, Y)

In [ ]:
def queryKmer(ID, isZoonotic_list, index, everything):
    FileName = "{}.gb".format(ID)
    try:
        QueryHandle = Entrez.efetch(db="nucleotide", id=ID, 
                                    rettype="gb", retmode="text")
    except HTTPError as Error:
        if Error.code == 400:  # Bad request
            raise ValueError(f"Accession number not found: {ID}")
        else:
            raise

    SeqRec = SeqIO.read(QueryHandle, "genbank")
    print(SeqRec.name)
    info = {'ID': ID, 'DNA Sequence': str(SeqRec.seq), 'isZoonotic': isZoonotic_list[index]}
    everything.append(info)

    pickle.dump(info, open(f"data/sequences/{ID}.pkl", "wb"))

In [ ]:
def getSequences(mergedDf):
    accession_list = [] # maintain order
    isZoonotic_list = [] # maintain order
    accession_set = set()
    isZoonotic_set = set()


    for row in tqdm.tqdm(mergedDf.itertuples()):
        # row[13] = accession, row[15] = infects humans
        for single_acc in row[14].split("; "):
            # print(single_acc)
            if single_acc not in accession_set:
                accession_list.append(single_acc)
                isZoonotic_list.append(0 if not row[16] else 1)
                # accession_set.add(single_acc)
                # isZoonotic_set.add(row[15])
                # print(0 if not row[16] else 1)

    print("passed local retrieval")
    # TODO: RUN MULTIPLE THREADS TO SPEED UP
    threads = []
    vals = []

    print(len(accession_list))
    for index, ID in enumerate(tqdm.tqdm(accession_list)): #only read the first 100 lol
        # multithread for speed up
        queryKmer(ID, isZoonotic_list, index, vals)
        # x = threading.Thread(target=queryKmer, args=(ID, isZoonotic_list, index, vals))
        # threads.append(x)
        # x.start()

    # for index, thread in enumerate(tqdm.tqdm(threads)):
    #     thread.join()
    df = pd.DataFrame(vals)
    df.to_csv("data/nardus_sequences.csv", index=False)

    return df
    


In [ ]:
# SLOW METHOD
mergedDf = pd.read_csv("data/FinalData_Cleaned.csv")
print(len(mergedDf))
sequences = getSequences(mergedDf)
(X_test, y_test) = getTrainParams(sequences, 4)

In [ ]:
print(len(X_test))
print(len(y_test))

In [ ]:
# FAST METHOD - LOAD DATA FROM FOLDER
li = []
for f in os.listdir('data/sequences'):
    seqdf = pickle.load(open(f'data/sequences/{f}', 'rb'))
    li.append(seqdf)
df = pd.DataFrame(li)
print(df)

# print(df)
(X_test_temp, y_test_temp) = getTrainParams(df, 4)


In [ ]:
best_gradBoost = pickle.load(open('models/curr_models/gradBoost.pkl', 'rb'))
best_xgBoost = pickle.load(open('models/curr_models/xgBoost.pkl', 'rb'))

kmer = 4
s = product('acgt',repeat = kmer)
permset = set(["".join(x) for x in list(s)])

pred_arr = []
xg_pred = []

# blood validation
for ind, file in enumerate(os.listdir("data/virome_contigs")):
    fasta_sequences = SeqIO.parse(open(f"data/virome_contigs/{file}"),'fasta')

    fasta = [x for x in fasta_sequences][0]
    # print(fasta)
    
    name, sequence = fasta.id, str(fasta.seq)
    X_info = sequence.lower()

    oDict = assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

    kmer_df = pd.DataFrame()

    for i in oDict:
        kmer_df.at[0, i]=oDict[i]

    kmer_df = kmer_df.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)
    
    cols_when_model_builds = best_gradBoost.feature_names_in_
    kmer_df=kmer_df[cols_when_model_builds]
    
    # print(kmer_df.to_string())
    
    pred_arr.append(best_gradBoost.predict(kmer_df))
    
    cols_when_model_builds = best_xgBoost.get_booster().feature_names
    kmer_df=kmer_df[cols_when_model_builds]
    # print(kmer_df.to_string())

    xg_pred.append(best_xgBoost.predict(kmer_df))
    
pred_arr = np.asarray(pred_arr)
xg_pred = np.asarray(xg_pred)


    # print(best_gradBoost.predict(kmer_df), sequences.loc[ind]['isZoonotic'])
    # print(best_gradBoost.predict(kmer_df), sequences['isZoonotic'])
        # print(accuracy_score())

In [ ]:
print(len(X_test_temp), len(y_test_temp))

In [ ]:
cols_when_model_builds = best_gradBoost.feature_names_in_
X_test_temp=X_test_temp[cols_when_model_builds]
print(len(X_test_temp))
print(accuracy_score(y_test_temp, best_gradBoost.predict(X_test_temp)))
print(confusion_matrix(y_test_temp, best_gradBoost.predict(X_test_temp)).ravel())

# print(y_test.to_string())

cols_when_model_builds = best_xgBoost.get_booster().feature_names
X_test_temp=X_test_temp[cols_when_model_builds]
print(accuracy_score(y_test_temp, best_xgBoost.predict(X_test_temp)))
print(confusion_matrix(y_test_temp, best_gradBoost.predict(X_test_temp)).ravel())


In [ ]:
print(X_test_temp)

In [ ]:
orig_df = pd.read_csv('data/info.csv')
X_vals = pd.concat([orig_df.loc[:, orig_df.columns != 'isZoonotic'], X_test_temp], axis=0, ignore_index=True)

# # print(X_vals)
y_vals = pd.concat([orig_df['isZoonotic'], y_test_temp], axis=0, ignore_index=True)

X_train, X_test, y_train, y_test = train_test_split(X_vals, y_vals, test_size=0.2, random_state=1)

# print(len(X_train)+len(X_test))

In [ ]:
lr_list = [0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65,0.7, 0.75, 0.8, 0.85, 1]

parameters={
   "n_estimators":130, # 200 kind of overfits I think
    "max_features":2,
    "max_depth":9,
    "random_state":0,
    "min_sample_split":25,
    "subsample":0.85
    # "warm_start":True
}


# careful with WARM START - only works after a lot of iterations
# integrating both datasets works better!

print(len(X_train)+len(X_test))
for learning_rate in lr_list:
    gradBoost = GradientBoostingClassifier(n_estimators=parameters['n_estimators'], 
    learning_rate=learning_rate, max_features=parameters['max_features'], 
    max_depth=parameters['max_depth'], random_state=parameters['random_state'], 
    min_samples_split=parameters['min_sample_split'], subsample=parameters['subsample']
    )

    parameters['learning_rate']=learning_rate
    gradBoost.fit(X_train, y_train)

    cols_when_model_builds = gradBoost.feature_names_in_
    X_test=X_test[cols_when_model_builds]
    
    testingAcc = accuracy_score(y_test, gradBoost.predict(X_test))
    trainingAcc = accuracy_score(y_train, gradBoost.predict(X_train))
    
    print("Learning rate: ", learning_rate)
    # print("Accuracy score (training): {0:.3f}".format(trainingAcc))
    print("Accuracy score (validation): {0:.3f}".format(testingAcc))
    # print(f"Feature importance {gradBoost.feature_importances_}")

    # pickle.dump(gradBoost, open('gradBoost.pkl', 'wb'))
    saveModel(gradBoost, "gradBoost", X_test, y_test, parameters, gradBoost=True, dir="models/nardus_testing")

In [ ]:
learning_rate = 0.05

parameters={
   "n_estimators":100, # 200 kind of overfits I think
    "max_features":2,
    "max_depth":9,
    "random_state":0,
    "min_sample_split":25,
    "subsample":0.85
    # "warm_start":True
}


param_test1 = {'n_estimators':range(100,150,10), 'learning_rate':[0.1,0.15, 0.2,0.25,0.3], 'subsample':[0.8,0.85,0.9], 'max_depth':range(6,10,1), 'min_samples_split':range(10,40,10)}

gradBoost = GridSearchCV(estimator = GradientBoostingClassifier(
    n_estimators=parameters['n_estimators'], 
learning_rate=learning_rate, max_features=parameters['max_features'], 
max_depth=parameters['max_depth'], random_state=parameters['random_state'], 
min_samples_split=parameters['min_sample_split'], subsample=parameters['subsample']
), 
param_grid = param_test1, scoring='roc_auc',n_jobs=-1, cv=5, verbose=10)

parameters['learning_rate']=learning_rate
gradBoost.fit(X_train, y_train)

pickle.dump(gradBoost, open('models/nardus_gridsearch.pkl', 'wb'))

In [ ]:
gradBoost = pickle.load(open('models/nardus_gridsearch.pkl', 'rb')).best_estimator_
print(len(X_test_temp))
cols_when_model_builds = gradBoost.feature_names_in_
X_test_temp=X_test_temp[cols_when_model_builds]

print(accuracy_score(y_test_temp, gradBoost.predict(X_test_temp)))
print(confusion_matrix(y_test_temp, gradBoost.predict(X_test_temp)).ravel())

In [ ]:
gradBoost = pickle.load(open('models/nardus_gridsearch.pkl', 'rb')).best_estimator_

cols_when_model_builds = gradBoost.feature_names_in_
X_test=X_test[cols_when_model_builds]

print(accuracy_score(y_test, gradBoost.predict(X_test)))
print(confusion_matrix(y_test, gradBoost.predict(X_test)).ravel())

In [ ]:
gradBoost = pickle.load(open('models/curr_models/gradBoost.pkl', 'rb'))
print(len(X_test_temp))
cols_when_model_builds = gradBoost.feature_names_in_
X_test_temp=X_test_temp[cols_when_model_builds]

print(accuracy_score(y_test_temp, gradBoost.predict(X_test_temp)))
print(confusion_matrix(y_test_temp, gradBoost.predict(X_test_temp)).ravel())

In [ ]:
gradBoost = pickle.load(open('models/nardus_gridsearch.pkl', 'rb')).best_estimator_
cols_when_model_builds = gradBoost.feature_names_in_
X_test_temp=X_test_temp[cols_when_model_builds]

print(accuracy_score(y_test_temp, gradBoost.predict(X_test_temp)).ravel())

print(confusion_matrix(y_test_temp, gradBoost.predict(X_test_temp)).ravel())

# ALWAYS reset X columns to the right order
y_thing = y_test_temp.to_numpy()
precision, recall, thresholds = precision_recall_curve(y_thing, gradBoost.predict_proba(X_test_temp)[::,1])
aaa = auc(recall, precision)

print("precision recall: " + str(aaa))

y_pred_proba = gradBoost.predict_proba(X_test_temp)[::,1]
fpr, tpr, _ = roc_curve(y_test_temp, y_pred_proba)
auc_thing = roc_auc_score(y_test_temp, y_pred_proba)

print(y_pred_proba)
plt.plot(fpr,tpr,label="AUC="+str(auc_thing))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
gradBoost = pickle.load(open('models/nardus_gridsearch.pkl', 'rb')).best_estimator_
cols_when_model_builds = gradBoost.feature_names_in_
X_test=X_test[cols_when_model_builds]

print(accuracy_score(y_test, gradBoost.predict(X_test)).ravel())

print(confusion_matrix(y_test, gradBoost.predict(X_test)).ravel())

# ALWAYS reset X columns to the right order
y_thing = y_test.to_numpy()
precision, recall, thresholds = precision_recall_curve(y_thing, gradBoost.predict_proba(X_test)[::,1])
aaa = auc(recall, precision)

print("precision recall: " + str(aaa))

y_pred_proba = gradBoost.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
auc_thing = roc_auc_score(y_test, y_pred_proba)

print(y_pred_proba)
plt.plot(fpr,tpr,label="AUC="+str(auc_thing))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
lr_list = [0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65,0.7, 0.75, 0.8, 0.85, 1]

parameters={
   "n_estimators":100, # 200 kind of overfits I think
    "max_features":2,
    "max_depth":8,
    "random_state":0,
    "subsample":0.8,
    'lambda': 0.5, # regularization?
    'alpha': 0.5
}



for learning_rate in lr_list:
    xgBoost = XGBClassifier(n_estimators=parameters['n_estimators'], 
    learning_rate=learning_rate, 
    max_depth=parameters['max_depth'], random_state=parameters['random_state'], 
    subsample=parameters['subsample'], 
    )

    parameters['learning_rate']=learning_rate
    xgBoost.fit(X_train, y_train)

    # ALWAYS reset feature names
    cols_when_model_builds = xgBoost.get_booster().feature_names
    X_test=X_test[cols_when_model_builds]

    testingAcc = accuracy_score(y_test, xgBoost.predict(X_test))
    trainingAcc = accuracy_score(y_train, xgBoost.predict(X_train))
    
    print("Learning rate: ", learning_rate)
    # print("Accuracy score (training): {0:.3f}".format(trainingAcc))
    print("Accuracy score (validation): {0:.3f}".format(testingAcc))
    # print(f"Feature importance {gradBoost.feature_importances_}")

    # pickle.dump(gradBoost, open('gradBoost.pkl', 'wb'))
    # saveModel(xgBoost, "xgBoost", X_test, y_test, parameters, xgBoost=True, dir="synthetic_data_testing")

In [ ]:
randforest = pickle.load(open('curr_models/randforest.pkl', 'rb'))
print(accuracy_score(y_train, randforest.predict(X_train)))

In [ ]:
knn = pickle.load(open('curr_models/knn.pkl', 'rb'))

print(accuracy_score(y_test, knn.predict(X_test)))
# knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
def getSingleSequence(accessionID):
    try:
        QueryHandle = Entrez.efetch(db="nucleotide", id=accessionID, 
                                    rettype="gb", retmode="text")
    except HTTPError as Error:
        if Error.code == 400:  # Bad request
            raise ValueError(f"Accession number not found: {accessionID}")
        else:
            raise

    SeqRec = SeqIO.read(QueryHandle, "genbank")
    print(str(SeqRec))

    oDict = assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

    kmer_df = pd.DataFrame()

    for i in oDict:
        kmer_df.at[0, i]=oDict[i]
    # print(best_gradBoost.predict_proba(kmer_df))

    cols_when_model_builds = best_gradBoost.feature_names_in_
    kmer_df=kmer_df[cols_when_model_builds]
    
    print([round(x, 2) for x in best_gradBoost.predict_proba(kmer_df).tolist()[0]])
    print(best_gradBoost.predict(kmer_df).tolist()[0])

    # ls = []
    # ls.append({'accession': accessionID, 'sequence': str(SeqRec.seq).lower()})
    # df = pd.DataFrame(ls)
    # print(df)

getSingleSequence("PA544053")